In [3]:
import os 
import gc
import itertools
import numpy as np
from tqdm import tqdm
from constants import *
from helper.models import *
from helper.modules import *
from helper.helpers import *
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix

In [4]:
def train_evaluate(labels, add_noise, oversampling, undersampling):
    config = {
        "data_path": DATA_PATH,
        "preprocess": True,
        "generate_images": True,
        "rows_per_person": 23,
        "frequency": 178,
        "labels": labels,
        "add_noise": add_noise,
        "oversampling": oversampling,
        "undersampling": undersampling,
        "test_size": 0.3
    }

    es = EpilepticSeizure(config)

    X_train, y_train, X_test, y_test = es.process()

    X_train_signals = X_train.iloc[:, :178].values
    X_train_signals = X_train_signals.reshape((X_train_signals.shape[0], 1, X_train_signals.shape[1]))

    X_test_signals = X_test.iloc[:, :178].values
    X_test_signals = X_test_signals.reshape((X_test_signals.shape[0], 1, X_test_signals.shape[1]))

    X_train_images = np.array([load_and_preprocess_image(path) for path in tqdm(X_train['Image_Path'])])
    X_test_images = np.array([load_and_preprocess_image(path) for path in tqdm(X_test['Image_Path'])])

    y_train = y_train['Label'].values
    y_test = y_test['Label'].values

    multi_head_model = MultiModal_Model(learning_rate=0.0001, dropout_rate_cnn=0.5, dropout_rate_lstm=0.2, labels=labels)

    history = multi_head_model.fit(
        [X_train_signals, X_train_images],
        to_categorical(y_train),
        shuffle=True,
        validation_data=(
            [X_test_signals, X_test_images],
            to_categorical(y_test)
        ),
        epochs=50,
        batch_size=46,
        callbacks=[lr_scheduler, early_stopping, gc_epoch]
    )

    plot_and_save(history, 'accuracy', 'Model Accuracy', f'accuracy_plot_{labels}_{add_noise}_{oversampling}_{undersampling}.png')
    plot_and_save(history, 'loss', 'Model Loss', f'loss_plot_{labels}_{add_noise}_{oversampling}_{undersampling}.png')

    scoreTrain, accTrain = multi_head_model.evaluate([X_train_signals, X_train_images], to_categorical(y_train), batch_size=23)
    print('Training Accuracy:', round(accTrain*100, 2), '%')

    scoreTest, accTest = multi_head_model.evaluate([X_test_signals, X_test_images], to_categorical(y_test), batch_size=23)
    print('Test Accuracy:', round(accTest*100, 2), '%')
    
    y_test_pred = np.argmax(multi_head_model.predict([X_test_signals, X_test_images]), axis=1)

    class_report = classification_report(y_test, y_test_pred)
    conf_matrix = confusion_matrix(y_test, y_test_pred)

    results = {
        "labels": labels,
        "add_noise": add_noise,
        "oversampling": oversampling,
        "undersampling": undersampling,
        "train_accuracy": round(accTrain*100, 2),
        "test_accuracy": round(accTest*100, 2),
        "train_loss": scoreTrain,
        "test_loss": scoreTest,
        "classification_report": class_report,
        "confusion_matrix": conf_matrix.tolist()
    }
    
    with open(f'results_{labels}_{add_noise}_{oversampling}_{undersampling}.txt', 'w') as f:
        for key, value in results.items():
            if key in ["classification_report", "confusion_matrix"]:
                f.write(f"{key}:\n{value}\n")
            else:
                f.write(f"{key}: {value}\n")

    return history, results

# 2 Labels Training

In [5]:
labels_list = [2]
add_noise_list = [True, False]
oversampling_list = [True, False]
undersampling_list = [True, False]

for labels, add_noise, oversampling, undersampling in itertools.product(labels_list, add_noise_list, oversampling_list, undersampling_list):
    print(f"Running for labels={labels}, add_noise={add_noise}, oversampling={oversampling}, undersampling={undersampling}")
    history, results = train_evaluate(labels, add_noise, oversampling, undersampling)
    gc.collect()

Running for labels=2, add_noise=True, oversampling=True, undersampling=True


100%|██████████| 3450/3450 [00:10<00:00, 320.37it/s]


Epoch 1/50
700/700 [==============================] - 27s 27ms/step - loss: 0.5051 - accuracy: 0.9389 - val_loss: 0.3366 - val_accuracy: 0.9649 - lr: 1.0000e-04
Epoch 2/50
700/700 [==============================] - 17s 24ms/step - loss: 0.2484 - accuracy: 0.9772 - val_loss: 0.2063 - val_accuracy: 0.9742 - lr: 1.0000e-04
Epoch 3/50
700/700 [==============================] - 17s 24ms/step - loss: 0.1514 - accuracy: 0.9836 - val_loss: 0.1586 - val_accuracy: 0.9742 - lr: 1.0000e-04
Epoch 4/50
700/700 [==============================] - 18s 25ms/step - loss: 0.0982 - accuracy: 0.9869 - val_loss: 0.1298 - val_accuracy: 0.9710 - lr: 1.0000e-04
Epoch 5/50
700/700 [==============================] - 18s 26ms/step - loss: 0.0706 - accuracy: 0.9885 - val_loss: 0.1657 - val_accuracy: 0.9661 - lr: 1.0000e-04
Epoch 6/50
700/700 [==============================] - 17s 25ms/step - loss: 0.0546 - accuracy: 0.9907 - val_loss: 0.0970 - val_accuracy: 0.9754 - lr: 9.0484e-05
Epoch 7/50
700/700 [==============

100%|██████████| 3450/3450 [00:11<00:00, 303.52it/s]


Epoch 1/50
631/631 [==============================] - 22s 30ms/step - loss: 0.5537 - accuracy: 0.9191 - val_loss: 0.3533 - val_accuracy: 0.9652 - lr: 1.0000e-04
Epoch 2/50
631/631 [==============================] - 19s 30ms/step - loss: 0.2865 - accuracy: 0.9731 - val_loss: 0.2930 - val_accuracy: 0.9597 - lr: 1.0000e-04
Epoch 3/50
631/631 [==============================] - 18s 29ms/step - loss: 0.1861 - accuracy: 0.9813 - val_loss: 0.2273 - val_accuracy: 0.9586 - lr: 1.0000e-04
Epoch 4/50
631/631 [==============================] - 18s 28ms/step - loss: 0.1280 - accuracy: 0.9847 - val_loss: 0.1847 - val_accuracy: 0.9594 - lr: 1.0000e-04
Epoch 5/50
631/631 [==============================] - 17s 27ms/step - loss: 0.0922 - accuracy: 0.9870 - val_loss: 0.1464 - val_accuracy: 0.9757 - lr: 1.0000e-04
Epoch 6/50
631/631 [==============================] - 18s 28ms/step - loss: 0.0711 - accuracy: 0.9888 - val_loss: 0.1715 - val_accuracy: 0.9600 - lr: 9.0484e-05
Epoch 7/50
631/631 [==============

...

Epoch 1/50
458/458 [==============================] - 18s 33ms/step - loss: 0.5786 - accuracy: 0.9104 - val_loss: 0.4974 - val_accuracy: 0.9548 - lr: 1.0000e-04
Epoch 2/50
458/458 [==============================] - 14s 30ms/step - loss: 0.3053 - accuracy: 0.9838 - val_loss: 0.3439 - val_accuracy: 0.9525 - lr: 1.0000e-04
Epoch 3/50
458/458 [==============================] - 14s 30ms/step - loss: 0.2014 - accuracy: 0.9922 - val_loss: 0.2295 - val_accuracy: 0.9632 - lr: 1.0000e-04
Epoch 4/50
458/458 [==============================] - 14s 30ms/step - loss: 0.1372 - accuracy: 0.9949 - val_loss: 0.1647 - val_accuracy: 0.9722 - lr: 1.0000e-04
Epoch 5/50
458/458 [==============================] - 14s 30ms/step - loss: 0.0951 - accuracy: 0.9958 - val_loss: 0.1798 - val_accuracy: 0.9638 - lr: 1.0000e-04
Epoch 6/50
458/458 [==============================] - 14s 30ms/step - loss: 0.0660 - accuracy: 0.9975 - val_loss: 0.1768 - val_accuracy: 0.9661 - lr: 9.0484e-05
Epoch 7/50
458/458 [==============

100%|██████████| 3450/3450 [00:10<00:00, 319.75it/s]


Epoch 1/50
241/241 [==============================] - 14s 48ms/step - loss: 0.7802 - accuracy: 0.8163 - val_loss: 0.6437 - val_accuracy: 0.9591 - lr: 1.0000e-04
Epoch 2/50
241/241 [==============================] - 11s 46ms/step - loss: 0.4277 - accuracy: 0.9657 - val_loss: 0.4778 - val_accuracy: 0.9696 - lr: 1.0000e-04
Epoch 3/50
241/241 [==============================] - 12s 50ms/step - loss: 0.3172 - accuracy: 0.9840 - val_loss: 0.2832 - val_accuracy: 0.9861 - lr: 1.0000e-04
Epoch 4/50
241/241 [==============================] - 12s 47ms/step - loss: 0.2541 - accuracy: 0.9861 - val_loss: 0.2435 - val_accuracy: 0.9814 - lr: 1.0000e-04
Epoch 5/50
241/241 [==============================] - 12s 47ms/step - loss: 0.1985 - accuracy: 0.9936 - val_loss: 0.1939 - val_accuracy: 0.9841 - lr: 1.0000e-04
Epoch 6/50
241/241 [==============================] - 11s 44ms/step - loss: 0.1625 - accuracy: 0.9943 - val_loss: 0.1780 - val_accuracy: 0.9823 - lr: 9.0484e-05
Epoch 7/50
241/241 [==============

100%|██████████| 3450/3450 [00:10<00:00, 324.26it/s]


Epoch 1/50
175/175 [==============================] - 13s 62ms/step - loss: 0.8686 - accuracy: 0.7812 - val_loss: 0.7754 - val_accuracy: 0.9113 - lr: 1.0000e-04
Epoch 2/50
175/175 [==============================] - 10s 56ms/step - loss: 0.5109 - accuracy: 0.9513 - val_loss: 0.5158 - val_accuracy: 0.9533 - lr: 1.0000e-04
Epoch 3/50
175/175 [==============================] - 10s 56ms/step - loss: 0.3909 - accuracy: 0.9737 - val_loss: 0.3906 - val_accuracy: 0.9675 - lr: 1.0000e-04
Epoch 4/50
175/175 [==============================] - 11s 65ms/step - loss: 0.3255 - accuracy: 0.9822 - val_loss: 0.3243 - val_accuracy: 0.9699 - lr: 1.0000e-04
Epoch 5/50
175/175 [==============================] - 11s 64ms/step - loss: 0.2688 - accuracy: 0.9907 - val_loss: 0.3127 - val_accuracy: 0.9643 - lr: 1.0000e-04
Epoch 6/50
175/175 [==============================] - 12s 65ms/step - loss: 0.2306 - accuracy: 0.9919 - val_loss: 0.2656 - val_accuracy: 0.9719 - lr: 9.0484e-05
Epoch 7/50
175/175 [==============

# 3 Labels Training

In [6]:
labels_list = [3]
add_noise_list = [True, False]
oversampling_list = [True, False]
undersampling_list = [True, False]

for labels, add_noise, oversampling, undersampling in itertools.product(labels_list, add_noise_list, oversampling_list, undersampling_list):
    print(f"Running for labels={labels}, add_noise={add_noise}, oversampling={oversampling}, undersampling={undersampling}")
    history, results = train_evaluate(labels, add_noise, oversampling, undersampling)
    gc.collect()

Running for labels=3, add_noise=True, oversampling=True, undersampling=True


100%|██████████| 3450/3450 [00:11<00:00, 296.66it/s]


Epoch 1/50
670/670 [==============================] - 69s 98ms/step - loss: 0.8958 - accuracy: 0.7682 - val_loss: 0.5291 - val_accuracy: 0.9139 - lr: 1.0000e-04
Epoch 2/50
670/670 [==============================] - 18s 27ms/step - loss: 0.5030 - accuracy: 0.8838 - val_loss: 0.3472 - val_accuracy: 0.9354 - lr: 1.0000e-04
Epoch 3/50
670/670 [==============================] - 18s 26ms/step - loss: 0.3731 - accuracy: 0.9056 - val_loss: 0.2856 - val_accuracy: 0.9510 - lr: 1.0000e-04
Epoch 4/50
670/670 [==============================] - 18s 27ms/step - loss: 0.2990 - accuracy: 0.9203 - val_loss: 0.3732 - val_accuracy: 0.9101 - lr: 1.0000e-04
Epoch 5/50
670/670 [==============================] - 18s 27ms/step - loss: 0.2489 - accuracy: 0.9307 - val_loss: 0.2303 - val_accuracy: 0.9394 - lr: 1.0000e-04
Epoch 6/50
670/670 [==============================] - 18s 26ms/step - loss: 0.2114 - accuracy: 0.9423 - val_loss: 0.2955 - val_accuracy: 0.9235 - lr: 9.0484e-05
Epoch 7/50
670/670 [==============

100%|██████████| 3450/3450 [00:12<00:00, 271.83it/s]


Epoch 1/50
560/560 [==============================] - 18s 26ms/step - loss: 0.9330 - accuracy: 0.7465 - val_loss: 0.9601 - val_accuracy: 0.7087 - lr: 1.0000e-04
Epoch 2/50
560/560 [==============================] - 13s 24ms/step - loss: 0.5440 - accuracy: 0.8696 - val_loss: 0.4041 - val_accuracy: 0.9322 - lr: 1.0000e-04
Epoch 3/50
560/560 [==============================] - 13s 24ms/step - loss: 0.4208 - accuracy: 0.8912 - val_loss: 0.3103 - val_accuracy: 0.9409 - lr: 1.0000e-04
Epoch 4/50
560/560 [==============================] - 13s 24ms/step - loss: 0.3441 - accuracy: 0.9081 - val_loss: 0.2806 - val_accuracy: 0.9455 - lr: 1.0000e-04
Epoch 5/50
560/560 [==============================] - 13s 24ms/step - loss: 0.2811 - accuracy: 0.9234 - val_loss: 0.2511 - val_accuracy: 0.9458 - lr: 1.0000e-04
Epoch 6/50
560/560 [==============================] - 13s 24ms/step - loss: 0.2423 - accuracy: 0.9337 - val_loss: 0.2203 - val_accuracy: 0.9490 - lr: 9.0484e-05
Epoch 7/50
560/560 [==============

...

Epoch 1/50
391/391 [==============================] - 16s 35ms/step - loss: 0.9041 - accuracy: 0.8050 - val_loss: 1.4452 - val_accuracy: 0.5696 - lr: 1.0000e-04
Epoch 2/50
391/391 [==============================] - 13s 33ms/step - loss: 0.3945 - accuracy: 0.9715 - val_loss: 0.5491 - val_accuracy: 0.9186 - lr: 1.0000e-04
Epoch 3/50
391/391 [==============================] - 13s 32ms/step - loss: 0.2789 - accuracy: 0.9860 - val_loss: 0.6162 - val_accuracy: 0.9093 - lr: 1.0000e-04
Epoch 4/50
391/391 [==============================] - 13s 32ms/step - loss: 0.2044 - accuracy: 0.9928 - val_loss: 0.3546 - val_accuracy: 0.9594 - lr: 1.0000e-04
Epoch 5/50
391/391 [==============================] - 13s 32ms/step - loss: 0.1546 - accuracy: 0.9950 - val_loss: 0.3237 - val_accuracy: 0.9557 - lr: 1.0000e-04
Epoch 6/50
391/391 [==============================] - 13s 32ms/step - loss: 0.1186 - accuracy: 0.9968 - val_loss: 0.2608 - val_accuracy: 0.9614 - lr: 9.0484e-05
Epoch 7/50
391/391 [==============

100%|██████████| 3450/3450 [00:10<00:00, 330.00it/s]


Epoch 1/50
291/291 [==============================] - 17s 47ms/step - loss: 0.9782 - accuracy: 0.7765 - val_loss: 1.4560 - val_accuracy: 0.4814 - lr: 1.0000e-04
Epoch 2/50
291/291 [==============================] - 13s 43ms/step - loss: 0.5061 - accuracy: 0.9514 - val_loss: 1.2681 - val_accuracy: 0.6284 - lr: 1.0000e-04
Epoch 3/50
291/291 [==============================] - 13s 44ms/step - loss: 0.3818 - accuracy: 0.9720 - val_loss: 0.3756 - val_accuracy: 0.9658 - lr: 1.0000e-04
Epoch 4/50
291/291 [==============================] - 13s 44ms/step - loss: 0.3050 - accuracy: 0.9811 - val_loss: 0.3161 - val_accuracy: 0.9681 - lr: 1.0000e-04
Epoch 5/50
291/291 [==============================] - 13s 43ms/step - loss: 0.2519 - accuracy: 0.9855 - val_loss: 0.2951 - val_accuracy: 0.9574 - lr: 1.0000e-04
Epoch 6/50
291/291 [==============================] - 13s 43ms/step - loss: 0.2075 - accuracy: 0.9899 - val_loss: 0.2675 - val_accuracy: 0.9670 - lr: 9.0484e-05
Epoch 7/50
291/291 [==============

100%|██████████| 3450/3450 [00:10<00:00, 343.15it/s]


Epoch 1/50
175/175 [==============================] - 13s 59ms/step - loss: 1.1624 - accuracy: 0.7058 - val_loss: 1.3770 - val_accuracy: 0.5928 - lr: 1.0000e-04
Epoch 2/50
175/175 [==============================] - 10s 56ms/step - loss: 0.6605 - accuracy: 0.9127 - val_loss: 1.2325 - val_accuracy: 0.6214 - lr: 1.0000e-04
Epoch 3/50
175/175 [==============================] - 10s 56ms/step - loss: 0.4981 - accuracy: 0.9463 - val_loss: 0.6586 - val_accuracy: 0.9107 - lr: 1.0000e-04
Epoch 4/50
175/175 [==============================] - 10s 56ms/step - loss: 0.4027 - accuracy: 0.9684 - val_loss: 0.4618 - val_accuracy: 0.9551 - lr: 1.0000e-04
Epoch 5/50
175/175 [==============================] - 10s 55ms/step - loss: 0.3479 - accuracy: 0.9742 - val_loss: 0.4733 - val_accuracy: 0.9406 - lr: 1.0000e-04
Epoch 6/50
175/175 [==============================] - 10s 55ms/step - loss: 0.3015 - accuracy: 0.9825 - val_loss: 0.5487 - val_accuracy: 0.9043 - lr: 9.0484e-05
Epoch 7/50
175/175 [==============

# 5 Labels Training

In [7]:
labels_list = [5]
add_noise_list = [True, False]
oversampling_list = [True, False]
undersampling_list = [True, False]

for labels, add_noise, oversampling, undersampling in itertools.product(labels_list, add_noise_list, oversampling_list, undersampling_list):
    print(f"Running for labels={labels}, add_noise={add_noise}, oversampling={oversampling}, undersampling={undersampling}")
    history, results = train_evaluate(labels, add_noise, oversampling, undersampling)
    gc.collect()

Running for labels=5, add_noise=True, oversampling=True, undersampling=True


100%|██████████| 3450/3450 [00:12<00:00, 284.25it/s]


Epoch 1/50
695/695 [==============================] - 47s 63ms/step - loss: 1.4462 - accuracy: 0.5297 - val_loss: 1.0092 - val_accuracy: 0.7151 - lr: 1.0000e-04
Epoch 2/50
695/695 [==============================] - 16s 22ms/step - loss: 0.9110 - accuracy: 0.7282 - val_loss: 0.9392 - val_accuracy: 0.6829 - lr: 1.0000e-04
Epoch 3/50
695/695 [==============================] - 15s 21ms/step - loss: 0.7333 - accuracy: 0.7808 - val_loss: 0.6586 - val_accuracy: 0.7872 - lr: 1.0000e-04
Epoch 4/50
695/695 [==============================] - 14s 20ms/step - loss: 0.6208 - accuracy: 0.8146 - val_loss: 0.5941 - val_accuracy: 0.8209 - lr: 1.0000e-04
Epoch 5/50
695/695 [==============================] - 14s 20ms/step - loss: 0.5426 - accuracy: 0.8392 - val_loss: 0.6479 - val_accuracy: 0.8223 - lr: 1.0000e-04
Epoch 6/50
695/695 [==============================] - 14s 20ms/step - loss: 0.4752 - accuracy: 0.8605 - val_loss: 0.8389 - val_accuracy: 0.7328 - lr: 9.0484e-05
Epoch 7/50
695/695 [==============

100%|██████████| 3450/3450 [00:14<00:00, 244.83it/s]


Epoch 1/50
535/535 [==============================] - 16s 25ms/step - loss: 1.5807 - accuracy: 0.4835 - val_loss: 1.5280 - val_accuracy: 0.4435 - lr: 1.0000e-04
Epoch 2/50
535/535 [==============================] - 12s 23ms/step - loss: 1.0584 - accuracy: 0.6713 - val_loss: 0.9165 - val_accuracy: 0.7006 - lr: 1.0000e-04
Epoch 3/50
535/535 [==============================] - 12s 23ms/step - loss: 0.8530 - accuracy: 0.7358 - val_loss: 0.9716 - val_accuracy: 0.6852 - lr: 1.0000e-04
Epoch 4/50
535/535 [==============================] - 12s 22ms/step - loss: 0.7383 - accuracy: 0.7709 - val_loss: 0.8531 - val_accuracy: 0.7116 - lr: 1.0000e-04
Epoch 5/50
535/535 [==============================] - 12s 23ms/step - loss: 0.6498 - accuracy: 0.7989 - val_loss: 0.7420 - val_accuracy: 0.7235 - lr: 1.0000e-04
Epoch 6/50
535/535 [==============================] - 12s 23ms/step - loss: 0.5851 - accuracy: 0.8165 - val_loss: 0.7020 - val_accuracy: 0.7762 - lr: 9.0484e-05
Epoch 7/50
535/535 [==============

...

Epoch 1/50
365/365 [==============================] - 16s 36ms/step - loss: 1.5051 - accuracy: 0.5306 - val_loss: 1.8947 - val_accuracy: 0.3617 - lr: 1.0000e-04
Epoch 2/50
365/365 [==============================] - 12s 34ms/step - loss: 0.8993 - accuracy: 0.7465 - val_loss: 0.9743 - val_accuracy: 0.6345 - lr: 1.0000e-04
Epoch 3/50
365/365 [==============================] - 12s 33ms/step - loss: 0.6726 - accuracy: 0.8273 - val_loss: 0.6701 - val_accuracy: 0.7754 - lr: 1.0000e-04
Epoch 4/50
365/365 [==============================] - 12s 34ms/step - loss: 0.5421 - accuracy: 0.8726 - val_loss: 0.7372 - val_accuracy: 0.7548 - lr: 1.0000e-04
Epoch 5/50
365/365 [==============================] - 12s 33ms/step - loss: 0.4453 - accuracy: 0.9020 - val_loss: 0.7082 - val_accuracy: 0.7719 - lr: 1.0000e-04
Epoch 6/50
365/365 [==============================] - 12s 34ms/step - loss: 0.3663 - accuracy: 0.9282 - val_loss: 0.7558 - val_accuracy: 0.7710 - lr: 9.0484e-05
Epoch 7/50
365/365 [==============

100%|██████████| 3450/3450 [00:10<00:00, 329.01it/s]


Epoch 1/50
338/338 [==============================] - 18s 43ms/step - loss: 1.6838 - accuracy: 0.4467 - val_loss: 1.6609 - val_accuracy: 0.4101 - lr: 1.0000e-04
Epoch 2/50
338/338 [==============================] - 14s 40ms/step - loss: 0.9646 - accuracy: 0.7480 - val_loss: 0.9665 - val_accuracy: 0.6777 - lr: 1.0000e-04
Epoch 3/50
338/338 [==============================] - 14s 41ms/step - loss: 0.6832 - accuracy: 0.8356 - val_loss: 0.6806 - val_accuracy: 0.7826 - lr: 1.0000e-04
Epoch 4/50
338/338 [==============================] - 14s 41ms/step - loss: 0.5506 - accuracy: 0.8689 - val_loss: 0.6223 - val_accuracy: 0.8035 - lr: 1.0000e-04
Epoch 5/50
338/338 [==============================] - 14s 40ms/step - loss: 0.4667 - accuracy: 0.8935 - val_loss: 0.7025 - val_accuracy: 0.7742 - lr: 1.0000e-04
Epoch 6/50
338/338 [==============================] - 14s 40ms/step - loss: 0.4014 - accuracy: 0.9106 - val_loss: 0.5220 - val_accuracy: 0.8472 - lr: 9.0484e-05
Epoch 7/50
338/338 [==============

100%|██████████| 3450/3450 [00:10<00:00, 331.47it/s]


Epoch 1/50
175/175 [==============================] - 13s 59ms/step - loss: 1.7620 - accuracy: 0.4181 - val_loss: 1.7990 - val_accuracy: 0.4696 - lr: 1.0000e-04
Epoch 2/50
175/175 [==============================] - 10s 54ms/step - loss: 1.2536 - accuracy: 0.6405 - val_loss: 1.4928 - val_accuracy: 0.5380 - lr: 1.0000e-04
Epoch 3/50
175/175 [==============================] - 10s 55ms/step - loss: 0.9721 - accuracy: 0.7421 - val_loss: 1.1468 - val_accuracy: 0.6409 - lr: 1.0000e-04
Epoch 4/50
175/175 [==============================] - 10s 55ms/step - loss: 0.8170 - accuracy: 0.7935 - val_loss: 0.8692 - val_accuracy: 0.7699 - lr: 1.0000e-04
Epoch 5/50
175/175 [==============================] - 10s 55ms/step - loss: 0.7117 - accuracy: 0.8324 - val_loss: 0.7652 - val_accuracy: 0.7991 - lr: 1.0000e-04
Epoch 6/50
175/175 [==============================] - 10s 55ms/step - loss: 0.6308 - accuracy: 0.8591 - val_loss: 0.8016 - val_accuracy: 0.8130 - lr: 9.0484e-05
Epoch 7/50
175/175 [==============

# Move Results to it Folders

In [13]:
labels_list = [2, 3, 5]
loss_files = [file for file in os.listdir() if file.startswith('loss_plot')]
results_files = [file for file in os.listdir() if file.startswith('results')]
accuracy_files = [file for file in os.listdir() if file.startswith('accuracy_plot')]

for label in tqdm(labels_list, desc="Creating label folders"):
    label_folder = f"{label}-label Results"
    os.makedirs(label_folder, exist_ok=True)

for file in tqdm(accuracy_files, desc="Moving accuracy files"):
    for label in labels_list:
        if str(label) in file:
            label_folder = f"{label}-label Results"
            os.rename(file, os.path.join(label_folder, file))

for file in tqdm(loss_files, desc="Moving loss files"):
    for label in labels_list:
        if str(label) in file:
            label_folder = f"{label}-label Results"
            os.rename(file, os.path.join(label_folder, file))

for file in tqdm(results_files, desc="Moving results files"):
    for label in labels_list:
        if str(label) in file:
            label_folder = f"{label}-label Results"
            os.rename(file, os.path.join(label_folder, file))

Moving results files: 100%|██████████| 24/24 [00:00<00:00, 2393.50it/s]


# SBA7 EL 5eer OStaz Mekhaimar EL training DONEEEEEE